# Trace quality analysis

In [1]:
import numpy as np
import sqlite3 as lite
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns

from utils.db_utils import get_db_absolute_path
from database.queries import QUERY_FULL_RANK_TEST_GROUPED_A

In [2]:
# Setup sqlite connection
database = get_db_absolute_path()
con = lite.connect(database)

## TP - Test traces RMS, hue 

In [30]:
query1 = """
        SELECT
            Rank_Test.id,
            Test_Datasets.test_dataset AS test_dataset,
            Training_Datasets.training_dataset AS training_dataset,
            Environments.environment AS environment,
            Rank_Test.distance,
            Rank_Test.device,
            Additive_Noise_Methods.additive_noise_method 
                AS additive_noise_method,
            Additive_Noise_Methods.additive_noise_method_parameter_1 
                AS additive_noise_method_parameter_1,
            Additive_Noise_Methods.additive_noise_method_parameter_1_value 
                AS additive_noise_method_parameter_1_value,
            Additive_Noise_Methods.additive_noise_method_parameter_2 
                AS additive_noise_method_parameter_2,
            Additive_Noise_Methods.additive_noise_method_parameter_2_value 
                AS additive_noise_method_parameter_2_value,
            Denoising_Methods.denoising_method AS denoising_method,
            Denoising_Methods.denoising_method_parameter_1 
                AS denoising_method_parameter_1,
            Denoising_Methods.denoising_method_parameter_1_value 
                AS denoising_method_parameter_1_value,
            Trace_Metadata_Depth.mean_val AS RMS,
            Rank_Test.termination_point
        FROM
            Rank_Test
        LEFT JOIN Test_Datasets 
            ON Test_Datasets.id = Rank_Test.test_dataset_id
        LEFT JOIN Training_Datasets 
            ON Training_Datasets.id = Rank_Test.training_dataset_id
        LEFT JOIN Environments 
            ON Environments.id = Rank_Test.environment_id
        LEFT JOIN Training_Models 
            ON Training_Models.id = Rank_Test.training_model_id
        LEFT JOIN Additive_Noise_Methods 
            ON Additive_Noise_Methods.id = Rank_test.additive_noise_method_id
        LEFT JOIN Denoising_Methods 
            ON Denoising_Methods.id = Rank_Test.denoising_method_id

        LEFT JOIN Trace_Metadata_Depth
            ON Trace_Metadata_Depth.test_dataset_id = Rank_Test.test_dataset_id
        AND Trace_Metadata_Depth.training_dataset_id = Rank_Test.training_dataset_id
        AND Trace_Metadata_Depth.environment_id = Rank_Test.environment_id
        AND Trace_Metadata_Depth.distance = Rank_Test.distance
        AND Trace_Metadata_Depth.device = Rank_Test.device
;
"""

query2 = """
        SELECT 
            Trace_Metadata_Depth.test_dataset_id,
            Trace_Metadata_Depth.training_dataset_id,
            Trace_Metadata_Depth.environment_id,
            Trace_Metadata_Depth.distance,
            Trace_Metadata_Depth.device,
            Trace_Metadata_Depth.trace_process_id,
            Trace_Metadata_Depth.data_point_index,
            Trace_Metadata_Depth.mean_val,
            Rank_Test.termination_point
        FROM 
            Trace_Metadata_Depth
        LEFT JOIN Rank_Test
            ON Trace_Metadata_Depth.test_dataset_id = Rank_Test.test_dataset_id
        AND Trace_Metadata_Depth.training_dataset_id = Rank_Test.training_dataset_id
        AND Trace_Metadata_Depth.environment_id = Rank_Test.environment_id
        AND Trace_Metadata_Depth.distance = Rank_Test.distance
        AND Trace_Metadata_Depth.device = Rank_Test.device
;
"""
data = pd.read_sql_query(query1, con)
data

,id,test_dataset,training_dataset,environment,distance,device,additive_noise_method,additive_noise_method_parameter_1,additive_noise_method_parameter_1_value,additive_noise_method_parameter_2,additive_noise_method_parameter_2_value,denoising_method,denoising_method_parameter_1,denoising_method_parameter_1_value,RMS,termination_point
0,1,Wang_2021,Wang_2021-Cable,office_corridor,15.0,6,None,None,NaN,None,NaN,Moving Average Filter,N,3.0,None,845
1,2,Wang_2021,Wang_2021-Cable,office_corridor,15.0,6,None,None,NaN,None,NaN,Moving Average Filter,N,3.0,None,958
2,3,Wang_2021,Wang_2021-Cable,office_corridor,15.0,6,None,None,NaN,None,NaN,Moving Average Filter,N,3.0,None,778
3,4,Wang_2021,Wang_2021-Cable,office_corridor,15.0,6,None,None,NaN,None,NaN,Moving Average Filter,N,3.0,None,891
4,5,Wang_2021,Wang_2021-Cable,office_corridor,15.0,6,None,None,NaN,None,NaN,Moving Average Filter,N,3.0,None,904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12990,12991,Zedigh_2021,Wang_2021-Cable,office_corridor,2.0,9,Rayleigh,Mode,0.0276,None,NaN,None,None,NaN,None,791
12991,12992,Zedigh_2021,Wang_2021-Cable,office_corridor,2.0,9,Rayleigh,Mode,0.0276,None,NaN,None,None,NaN,None,899
12992,12993,Zedigh_2021,Wang_2021-Cable,office_corridor,2.0,9,Rayleigh,Mode,0.0276,None,NaN,None,None,NaN,None,775
12993,12994,Zedigh_2021,Wang_2021-Cable,office_corridor,2.0,9,Rayleigh,Mode,0.0276,None,NaN,None,None,NaN,None,801
